In [ ]:
#Load the data set:
#necessary imports
import torch
import pandas as pd 
from torch.utils.data import Dataset, DataLoader

#Load data set
dataset = pd.read_csv('eestec_hackathon_2025_train.tsv' ,sep = '\t',names=['ID', 'Label', 'Statement', 'Subjects', 'Speaker Name', 'Speaker Title', 'State', 'Party Affiliation', 'Credit History: barely-true', 'Credit History: false', 'Credit History: half-true', 'Credit History: mostly-true', 'Credit History: pants-fire', 'Context/Location'])



In [ ]:
import csv
import pandas as pd 
import torch
import torchtext 
from torchtext.data import Field, TabularDataset
#Define pre processing for the statement column 
#Statement = Field(sequential =  True, lower = True, tokenize = "spacy", stop_words = "english", batch_first = True)
#Define the Score field 
#Label_False = Field(sequential = False, use_vocab = False, is_Target = True, batch_first = True, dtype = torch.int16)
#Label_Barely_True =  Field(sequential = False, use_vocab = False, is_Target = True, batch_first = True, dtype = torch.int16)
#Label_Half_True =  Field(sequential = False, use_vocab = False, is_Target = True, batch_first = True, dtype = torch.int16)
#Label_Mostly_True =  Field(sequential = False, use_vocab = False, is_Target = True, batch_first = True, dtype = torch.int16)
#Label_Pants_Fire =  Field(sequential = False, use_vocab = False, is_Target = True, batch_first = True, dtype = torch.int16)



 


In [ ]:
import pandas as pd
import spacy
import torch
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # Optional for numerical features
import numpy as np

File_Path = 'eestec_hackathon_2025_train.tsv'
Text_Column = 'Statement'
Numerical_Columns = [
    'Credit History: barely-true', 'Credit History: false',
    'Credit History: half-true', 'Credit History: mostly-true',
    'Credit History: pants-fire'
]
Label_Column = 'Label'


#Batch size 
Batch_Size = 64

#Read file  
Data_File = pd.read_csv('eestec_hackathon_2025_train.tsv' ,sep = '\t',names=['ID', 'Label', 'Statement', 'Subjects', 'Speaker Name', 'Speaker Title', 'State', 'Party Affiliation', 'Credit History: barely-true', 'Credit History: false', 'Credit History: half-true', 'Credit History: mostly-true', 'Credit History: pants-fire', 'Context/Location'])

#Split up the data in the validation and training set
Train_df, Val_df = train_test_split(Data_File, test_size=0.2, random_state=42)


# Preprocess the text 
natural_language_processor = spacy.load("en_core_web_sm")

def tokenize_text(text):
    #Lemmatize every token in set, after tokenizaziotn 
    return [token.lemma_.lower() for token in natural_language_processor(str(text)) if not token.is_punct and not token.is_space]

#Now we build the vocabulary 
word_count = Counter()
for statement in Train_df[Text_Column]:
    word_count.update(tokenize_text(statement))

# add PAD token to make same length and UNK(unkown for words not in the vocab)
Max_Vocab_Size = 10000
Max_Length = 50  
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
vocab = {PAD_TOKEN: 0, UNK_TOKEN: 1}
for i, (word, count) in enumerate(word_count.most_common(Max_Vocab_Size-2)): # -2 for pad and unk
    vocab[word] = i + 2

actual_Vocab_Size = len[vocab]

#Numericalize and pad/truncuate text
def numericalize_pad_text(text, vocab_map, max_len):
    tokens = tokenize_text(text)
    numericalized = [vocab_map.get(token, vocab_map[UNK_TOKEN]) for token in tokens]
    if len(numericalized) < max_len:
        # Pad with PAD_TOKEN's index
        numericalized.extend([vocab_map[PAD_TOKEN]] * (max_len - len(numericalized)))
    else:
        # Truncate
        numericalized = numericalized[:max_len]
    return numericalized

# Apply to Statement column
Train_df['statement_numerical'] = Train_df[Text_Column].apply(lambda x: numericalize_pad_text(x, vocab, Max_Length))
Val_df['statement_numerical'] = Val_df[Text_Column].apply(lambda x: numericalize_pad_text(x, vocab, Max_Length))

# Later maybe use scalar?
# scaler = StandardScaler()
# Train_df[Numerical_Columns] = scaler.fit_transform(Train_df[Numerical_Columns])
# Val_df[Numerical_Columns] = scaler.transform(Val_df[Numerical_Columns]) # Use transform for validation

# Mapping the labels to numbers
unique_labels = Train_df[Label_Column].astype(str).unique()
label_to_idx = {label: i for i, label in enumerate(unique_labels)}
idx_to_label = {i: label for label, i in label_to_idx.items()}
num_classes = len(unique_labels)

# Use mapping for data 
Train_df['label_idx'] = Train_df[Label_Column].astype(str).map(label_to_idx)
Val_df['label_idx'] = Val_df[Label_Column].astype(str).map(label_to_idx)

# Verify pleasefgxh cgjvb
class TextNumericalDataset(Dataset):
    def __init__(self, dataframe, text_col_numerical, numerical_cols_list, label_col_idx):
        self.texts = torch.tensor(list(dataframe[text_col_numerical].values), dtype=torch.long)
        self.numerical_features = torch.tensor(dataframe[numerical_cols_list].values, dtype=torch.int16)
        self.labels = torch.tensor(dataframe[label_col_idx].values, dtype=torch.int8) # Assuming classification

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'text': self.texts[idx],
            'numerical': self.numerical_features[idx],
            'label': self.labels[idx]
        }

train_dataset = TextNumericalDataset(Train_df, 'statement_numerical', Numerical_Columns, 'label_idx')
val_dataset = TextNumericalDataset(Val_df, 'statement_numerical', Numerical_Columns, 'label_idx')